In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import konlpy.tag 
import pandas as pd
import scipy as sp
from scipy import spatial
import operator
import numpy as np

In [2]:
def get_datas(date, comment=1):
    """ get naver news articles and comments"""
    article_df = pd.read_csv("../data/news/" + date + ".csv")
    comment_df = pd.read_csv("../data/comment/comment"+str(comment)+".csv")
    return article_df, comment_df

In [3]:
def make_document_morpheme(texts):
    """make documents list after morphological"""
    
    documents = []
    
    komoran = konlpy.tag.Komoran()

    for text in texts:
        obj = line2obj(komoran.pos(text, flatten=False))
        documents.append(obj)
        
    return documents

def line2obj(lines):
    """ make keywords dictionary include only (NNP, NNG)"""
    
    obj = {}
    
    for line in lines:
        for keyword in line:
            if len(keyword[0]) > 1 and ( keyword[1] == "NNP" or keyword[1] == "NNG" ) :
                key = keyword[0]
                if key in obj:
                    obj[key] += 1
                else:
                    obj[key] = 1
                    
    return obj

In [4]:
def gen_text_documents(documents):
    
    text_documents = []
    
    for document in documents:
        
        text_words = []
        
        for word, count in document.items():
            text_words.extend([word] * count)
            
        text_document = " ".join(text_words)
        text_documents.append(text_document)
        
    return text_documents        

In [5]:
def calc_euc_dist(v1, v2):
    """ euclidean distance """
    
    delta = v1 - v2
    dist = sp.linalg.norm(delta.toarray())
    return dist

def calc_cos_dist(v1, v2):
    """ cosin distance """
#     v1 = v1.toarray()
#     v2 = v2.toarray()
#     dist = 1.0 - np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    dist = spatial.distance.cosine(v1.toarray(), v2.toarray())
    
    return dist

def get_euc_dists(vectorized, num_docs, comm="euc"):
    
    dists = []
    
    for idx in range(num_docs-1):
            
        if comm == "euc":
            dist = calc_euc_dist(vectorized.getrow(num_docs-1), vectorized.getrow(idx))
            dists.append((num_docs-1, idx, dist))
            
        else :
            dist = calc_cos_dist(vectorized.getrow(num_docs-1), vectorized.getrow(idx))
            dists.append((num_docs-1, idx, dist))
        
    return dists

def sort_dists(dists):
    result_list = []
    for i, j, dist in sorted(dists, key=operator.itemgetter(2)):
        result_list.append((i, j, dist))
    return result_list

In [6]:
# get data
article_df, comment_df = get_datas("2016-07-06")
len(article_df), len(comment_df)

(2241, 15)

In [7]:
# make documents
comment_str = comment_df["content"][:].str.cat(sep=' ')
comment_str

'영화 `로봇소리`많이 기대하고 있습니다. 폭풍 연기력을 보여주실 이성민 배우분의 연기력을 기대해봅니다. 우선 오늘 저녁8부터는 다시 항공편 운항 재개를 한다고 하니 조금만 더 기다리시면 될 것 같아요. 제주도 관계자분들도 힘내세요! 영화 `로봇소리`많이 기대하고 있습니다. 폭풍 연기력을 보여주실 이성민 배우분의 연기력을 기대해봅니다. 아울러 대구 지하철 참사를 다시 생각하게 해주셔서 감사합니다. 제주도를 비롯해서 다른 지역 충청이남서해안 지역분들과 전라도 부근 주민들도 힘내세요! 눈 때문에 고생이 많으세요! 그래도 자수를 했다니 다행이네요, 피해자분들도 마음 잘 추수리셔서 화이팅하셨으면 좋겠습니다. 경찰분들도 바쁘신건 알지만 힘 내주세요! 충북 제천지역도 다른지역과 같이 많이 추운가 봅니다ㅠ 농민분들이 엄청 고생하시는 것 같습니다. 조금만 있으면 날씨가 풀릴 것 같으니 힘내세요! 평창올림픽의 성공을 위하여 관련 관계자분들이 스페인 마드리드까지 가서 신다고 하는데 감사합니다. 평창올림픽의 성공적인 개최를 기원합니다. KTX 지연, 제주공항 운항 지연에 뒤이은 여러번의 동파사고까지, 조금만 지나면 날시가 풀릴테니 힘내세요!! 모쪼록 몸에 해로운 담배입니다. 다른 나라의 광고가 어떻든 대한민국 국민의 건강 증진을 위하여 금연을 기원하시는 분들은 금연에 성공하시길 바라겠습니다. 어린 아이들이 이렇게 모범을 보여주니 너무 고맙습니다. 화단정리에 노인들께 식사를 대접하기까지 너무 마음이 예쁩니다. 이렇게 노인장기요양서비스가 통합(통일)된다면 노인분들이 이용하시기에도 간편하고 좋은 서비스활동이 될 것 같습니다. 미래를 위해 방학중에 대학 공동 진로진학 박람회까지 가서 자신의 미래를 설꼐하는것을 보면 너무 아름다운 것 같습니다. 이 추운 날씨에도 고생해주시는 장병분들을 보면 감사한 마음 뿐입니다. 급식을 비롯해서 여러 여건들이 개선되길 희망합니다! 이렇게 추운날씨에 남들을 위해  쌀 1000kg을 기부해주신 대한불교조계종 문수사(주지 스님 혜안)께 너무 감사합니다! 이렇게

In [8]:
article_series = article_df["content"][2000:2240]
article_series[:5], article_series[-5:]

(2000    문을 닫은 버스 터미널의 주차장에 허름한 천막이 세워져 있습니다. 재활용 의류 장터...
 2001    【런던=AP/뉴시스】영국 중앙은행인 영국은행의 마크 카니 총재가 5일 런던에서 기자...
 2002    [HOOC=서상범 기자ㆍ이영돈 인턴기자] 마블 코믹스 히어로들과 함께 웃고 있는 이...
 2003    [헤럴드경제] 유럽연합(EU)에서 또 다시 긴장감이 고조되고 있다. 영국 일간 파이...
 2004    1차 투표서 여성 후보 1, 2위... 26년 만의 '여성 총리' 유력[오마이뉴스윤...
 Name: content, dtype: object,
 2235    SK텔레콤과 CJ헬로비전의 인수합병이 무산될 가능성이 커졌다. 공정거래위원회 사무처...
 2236    SK텔레콤의 자회사인 SK브로드밴드와 CJ헬로비전의 합병 인가 여부는 이달 중 열리...
 2237    공정거래위원회는 SK텔레콤(SKT)과 CJ헬로비전의 인수를 불허한 이유를 ‘권역별 ...
 2238    미국 항공우주국(NASA)이 발사한 목성 탐사선 주노(Juno)가 5일 오후 목성 ...
 2239    7일 본입찰…매각가 4조~5조[ 이동훈 기자  ] CJ E&M이 2대주주인 국내 온...
 Name: content, dtype: object)

In [9]:
article_series = article_series.append(pd.Series([comment_str])).reset_index(drop=True)
article_series[:5], article_series[-5:]

(0    문을 닫은 버스 터미널의 주차장에 허름한 천막이 세워져 있습니다. 재활용 의류 장터...
 1    【런던=AP/뉴시스】영국 중앙은행인 영국은행의 마크 카니 총재가 5일 런던에서 기자...
 2    [HOOC=서상범 기자ㆍ이영돈 인턴기자] 마블 코믹스 히어로들과 함께 웃고 있는 이...
 3    [헤럴드경제] 유럽연합(EU)에서 또 다시 긴장감이 고조되고 있다. 영국 일간 파이...
 4    1차 투표서 여성 후보 1, 2위... 26년 만의 '여성 총리' 유력[오마이뉴스윤...
 dtype: object, 236    SK텔레콤의 자회사인 SK브로드밴드와 CJ헬로비전의 합병 인가 여부는 이달 중 열리...
 237    공정거래위원회는 SK텔레콤(SKT)과 CJ헬로비전의 인수를 불허한 이유를 ‘권역별 ...
 238    미국 항공우주국(NASA)이 발사한 목성 탐사선 주노(Juno)가 5일 오후 목성 ...
 239    7일 본입찰…매각가 4조~5조[ 이동훈 기자  ] CJ E&M이 2대주주인 국내 온...
 240    영화 `로봇소리`많이 기대하고 있습니다. 폭풍 연기력을 보여주실 이성민 배우분의 연...
 dtype: object)

In [10]:
%time article_documents = make_document_morpheme(article_series)

CPU times: user 22.2 s, sys: 516 ms, total: 22.8 s
Wall time: 9.81 s


In [11]:
# make gen documents
gen_articles = gen_text_documents(article_documents)
gen_articles[0]

'시내 시내 알리 알리 신발 배포 두 아들 악화 강제 이주 이번 경제 라마단 아랍의 봄 일자리 허브 한숨 기자 고가 한계 이집트 리드 인구 카이로 카이로 이제 상납 교통 체증 상인 상인 상인 전시 천막 장터 장터 장터 장터 장터 장터 걱정 물건 경찰 무단 처지 처지 아들 정부 정부 마음 당장 재활용 재활용 재활용 하루 교통 주차장 지대 우범 콘텐츠 다짐 생계 생계 마련 처음 처음 카말 정규 공무원 인적 사탕 단속 단속 불법 손님 유동 방해 도심 행정 복제 거리 거리 버스 터미널 장사 도로변 관광객 노점상 노점상 노점상 노점상 노점상 노점상 노점상 노점상 노점상 의류 의류 의류 금지 자리 노점 주차 뒷돈 이후 출신 빈말 사주'

In [12]:
# vectorizer = CountVectorizer()
# vectorizer = HashingVectorizer()
vectorizer = TfidfVectorizer()

vectorized = vectorizer.fit_transform(gen_articles)
num_docs, num_features =  vectorized.shape
feature_names = vectorizer.get_feature_names()

print("the number of input documents : {}".format(num_docs))
print("the number of words : {}".format(num_features))
result_df = pd.DataFrame(vectorized.toarray(), columns=feature_names)

the number of input documents : 241
the number of words : 7617


In [13]:
dists = get_euc_dists(vectorized, num_docs)
result = sort_dists(dists)
result[:5], result[-5:]

([(240, 33, 1.3267144813119582),
  (240, 225, 1.3366362247615911),
  (240, 138, 1.3782891855641217),
  (240, 107, 1.3796248109627118),
  (240, 102, 1.3798057153759107)],
 [(240, 17, 1.4142135623730951),
  (240, 32, 1.4142135623730951),
  (240, 77, 1.4142135623730951),
  (240, 147, 1.4142135623730951),
  (240, 224, 1.4142135623730951)])

In [14]:
dists = get_euc_dists(vectorized, num_docs, "cos")
result = sort_dists(dists)
result[:5], result[-5:]

([(240, 33, 0.88008565746142897),
  (240, 225, 0.89329819867245985),
  (240, 138, 0.94984053952150516),
  (240, 107, 0.9516823095119491),
  (240, 102, 0.95193190609201395)],
 [(240, 147, 1.0),
  (240, 153, 1.0),
  (240, 199, 1.0),
  (240, 224, 1.0),
  (240, 226, 1.0)])

In [15]:
# comment
print(comment_str)

영화 `로봇소리`많이 기대하고 있습니다. 폭풍 연기력을 보여주실 이성민 배우분의 연기력을 기대해봅니다. 우선 오늘 저녁8부터는 다시 항공편 운항 재개를 한다고 하니 조금만 더 기다리시면 될 것 같아요. 제주도 관계자분들도 힘내세요! 영화 `로봇소리`많이 기대하고 있습니다. 폭풍 연기력을 보여주실 이성민 배우분의 연기력을 기대해봅니다. 아울러 대구 지하철 참사를 다시 생각하게 해주셔서 감사합니다. 제주도를 비롯해서 다른 지역 충청이남서해안 지역분들과 전라도 부근 주민들도 힘내세요! 눈 때문에 고생이 많으세요! 그래도 자수를 했다니 다행이네요, 피해자분들도 마음 잘 추수리셔서 화이팅하셨으면 좋겠습니다. 경찰분들도 바쁘신건 알지만 힘 내주세요! 충북 제천지역도 다른지역과 같이 많이 추운가 봅니다ㅠ 농민분들이 엄청 고생하시는 것 같습니다. 조금만 있으면 날씨가 풀릴 것 같으니 힘내세요! 평창올림픽의 성공을 위하여 관련 관계자분들이 스페인 마드리드까지 가서 신다고 하는데 감사합니다. 평창올림픽의 성공적인 개최를 기원합니다. KTX 지연, 제주공항 운항 지연에 뒤이은 여러번의 동파사고까지, 조금만 지나면 날시가 풀릴테니 힘내세요!! 모쪼록 몸에 해로운 담배입니다. 다른 나라의 광고가 어떻든 대한민국 국민의 건강 증진을 위하여 금연을 기원하시는 분들은 금연에 성공하시길 바라겠습니다. 어린 아이들이 이렇게 모범을 보여주니 너무 고맙습니다. 화단정리에 노인들께 식사를 대접하기까지 너무 마음이 예쁩니다. 이렇게 노인장기요양서비스가 통합(통일)된다면 노인분들이 이용하시기에도 간편하고 좋은 서비스활동이 될 것 같습니다. 미래를 위해 방학중에 대학 공동 진로진학 박람회까지 가서 자신의 미래를 설꼐하는것을 보면 너무 아름다운 것 같습니다. 이 추운 날씨에도 고생해주시는 장병분들을 보면 감사한 마음 뿐입니다. 급식을 비롯해서 여러 여건들이 개선되길 희망합니다! 이렇게 추운날씨에 남들을 위해  쌀 1000kg을 기부해주신 대한불교조계종 문수사(주지 스님 혜안)께 너무 감사합니다! 이렇게 

In [16]:
comment, rank_idx, distance = zip(*result)
idx = rank_idx[0]
print(idx)
print(article_series[idx].split("▶")[0])

33
로사 코른펠드-마테 [유엔 홈페이지 캡처]노인이 브렉시트 희생양…인권 전문가 "민주사회 근본 위협" 우려      (서울=연합뉴스) 한미희 기자 = 영국에서 유럽연합(EU) 탈퇴(브렉시트) 국민투표 이후 노인을 향한 차별적인 공격이 난무해 유엔이 경계하고 나섰다.     유엔인권최고대표사무소(OHCHR)가 임명한 노인 인권 전문가인 로사 코른펠드-마테는 5일(현지시간) 유엔 홈페이지를 통해 "유럽의 많은 언론과 소셜미디어가 노인을 브렉시트의 희생양으로 삼고 이들의 투표권을 제한해야 한다고 요구하고 있다"고 전했다.     그러면서 "나이를 근거로 특정 권리의 행사를 제한하는 것은 국제 인권법상 용납할 수 없는 일"이라고 강조했다. 지난달 23일 브렉시트 투표소로 향하는 여성 노인 [AFP=연합뉴스]    지난달 23일 치러진 영국의 브렉시트 국민투표에서 65세 이상 노인들은 대거 탈퇴를 지지해 잔류를 원했던 젊은 EU 세대와의 극명한 격차를 보였다.     투표 결과에 대해 젊은이들은 '노인들이 우리의 미래를 망쳤다'고 반발하며 세대 갈등을 드러냈다.     코른펠드-마테는 "한 잡지는 일정 연령 이상이 되면 노인들의 운전면허를 박탈하듯 투표권을 빼앗아야 한다고 주장하고 있다"고 단적인 사례를 소개했다.     그는 "나이가 들었다는 이유로 그 사람의 정치적 권한에 이의를 제기해서는 안 된다"며 "공적 영역과 의사 결정 과정에서 모든 개인의 동등한 권리를 존중하는 것은 민주사회의 근본"이라고 말했다.     그러면서 "전 세계의 사회가 고령화하면서 세대 간 결속과 노인의 가치 있는 기여에 대한 사회적 이해를 높이기 위해 투자가 필요하다"고 목소리를 높였다. 제복입은 노인들의 투표행진(AFP=연합뉴스)     mihee@yna.co.kr


In [17]:
idx = len(rank_idx) - 1
print(article_series[idx].split("▶")[0])

7일 본입찰…매각가 4조~5조[ 이동훈 기자  ] CJ E&M이 2대주주인 국내 온라인게임 업체 넷마블게임즈가 모바일 카지노게임 세계 1위 업체 플레이티카(Playtika) 인수전에 뛰어들었다. 거래 가격이 4조~5조원에 이를 것으로 추정되는 대형 기업 인수합병(M&A)으로 중국 등 해외 업체와 인수 경쟁을 벌일 전망이다.5일 투자은행(IB)업계에 따르면 넷마블게임즈는 이스라엘에 본사를 둔 플레이티카 인수를 위해 최근 인수의향서를 제출했으며 7일 본입찰에 참여할 것으로 알려졌다. 거래금액 중 3조원 정도는 국내 기관투자가로부터 조달한다는 계획도 마련했다. 넷마블게임즈는 해외 대형 게임사를 인수한다는 방침을 세우고 올초 모건스탠리를 자문사로 선정해 인수 대상을 물색해왔다.플레이티카는 소셜네트워크서비스(SNS)나 스마트폰에서 포커 빙고 등 도박 게임을 즐기는 이른바 ‘소셜카지노’ 업체다. 페이스북, 구글 등에서 매출 상위권을 차지하고 있는 슬롯마니아, 시저스카지노 등이 이 회사가 개발한 게임이다. 2011년 미국 시저스인터랙티브엔터테인먼트(CIE)에 약 1억달러에 팔렸다.CIE 모회사인 세계 최대 카지노 업체 시저스엔터테인먼트가 재무 위기에 빠지면서 매물로 나왔지만 성장성은 무궁무진하다는 평가다. 지난해에는 7억달러(약 8000억원)에 육박하는 매출을 기록했으며 올해 1분기까지 전년 동기 대비 30%가량 매출이 늘어난 것으로 알려졌다. 넷마블게임즈 외에도 해외 유력 업체들이 대거 인수전에 참여할 전망이다. IB업계 관계자는 “중국과 동남아시아 시장 진출에 성공하면 기업가치가 훨씬 오를 수 있어서 중국 업체들이 높은 가격을 제시할 가능성이 크다”고 말했다.넷마블게임즈도 3대주주인 중국 인터넷 1위 업체 텐센트와 손잡고 중국 시장을 공략할 수 있다고 판단해 이번 거래에 참여한 것으로 보인다. 넷마블게임즈는 넷마블 창업자인 방준혁 의장이 32.36%로 최대주주다. CJ E&M과 텐센트가 각각 31.40%와 25.26%를 보유하고 있다.플레이티카 인수에 성공하면 내년을 